<a href="https://colab.research.google.com/github/aman-welzin/rag/blob/main/Experiment_YANA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain langchain-community langchain-weaviate gradio pymupdf python-docx ollama langchain-huggingface newspaper3k lxml-html-clean pillow langgraph

# Install Ollama and CUDA drivers
import os
!nvidia-smi
!curl https://ollama.ai/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

!nohup ollama serve &
!ollama ps
# !ollama pull mistral-small3.1
!ollama pull deepseek-r1:1.5b
!ollama list

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade --force-reinstall numpy pandas

In [1]:
import io
import os, uuid
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm
import json
import re
import urllib
import pprint
from IPython import embed
import matplotlib.font_manager
import matplotlib as mpl
import newspaper
from newspaper import Config
import sqlite3
import codecs
import base64
import PIL
from PIL import Image, ImageFont, ImageDraw, ImageColor
import textwrap
from IPython.display import Image, display

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from transformers import pipeline
import weaviate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.embedded import EmbeddedOptions
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import ParentDocumentRetriever
from google.colab import userdata

from langgraph.graph.message import add_messages
from langgraph.graph import END, StateGraph, START

from typing import List
from typing_extensions import TypedDict, Annotated

from pydantic import BaseModel, Field

import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [2]:
EMBEDDING_MODEL = "sentence-transformers/static-retrieval-mrl-en-v1"
MISTRAL_MODEL = "deepseek-r1:1.5b"

In [11]:
CURRENT_DATE = datetime.now().date()

drive.mount('/content/drive')

DB_PATH = '/content/drive/Shareddrives/Welzin/Yana/YanaNewsProd.db'

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

rows = cursor.execute(
    f"SELECT title,batch_dt, desc FROM clean_news_new"
).fetchall()
clean_news = pd.DataFrame(rows, columns=[description[0] for description in cursor.description])

Mounted at /content/drive


In [20]:
clean_news

,title,batch_dt,desc
0,PM speaks to Elon Musk ahead of Tesla's India ...,2025-04-18,Prime Minister Narendra Modi on Friday held a ...
1,Who is Phoenix Ikner? Florida State University...,2025-04-18,"Phoenix Ikner, the 20-year-old son of a sherif..."
2,Good Friday 2025: Are Sensex and Nifty closed ...,2025-04-18,Investors looking to continue Thursday’s rally...
3,"Trump's tariffs could hurt US, but an opportun...",2025-04-18,India could be one of the surprise beneficiari...
4,JEE Mains Result 2025 Live: Result latest by t...,2025-04-18,The National Testing Agency (NTA) is all set t...
...,...,...,...
1676,"The US's 2,000-year-old mystery mounds",2025-04-24,"The US's 2,000-year-old mystery mounds\n\n8 Ap..."
1677,Nine European vacation destinations to visit i...,2025-04-24,Nine European vacation destinations to visit i...
1678,A 58-tunnel slow train through India's Eastern...,2025-04-24,A 58-tunnel slow train through India's Eastern...
1679,The overlooked masterpiece full of coded messa...,2025-04-24,The overlooked masterpiece full of coded messa...


In [19]:
import weaviate.classes.config as wvc
from weaviate.collections.classes.config import Property, DataType

W_VERSION = "1.26.5"
with weaviate.connect_to_embedded(
    port                  = 8081,
    grpc_port             = 50051,
    version               = W_VERSION,
    persistence_data_path = "/content/weaviate_data",
) as client:
    assert client.is_ready(), "Weaviate did not start!"

    # 2) Define your schema (if not already present)
    #    This creates a collection/class named "Article" with 3 properties.
    if not client.collections.exists("Article"):
        client.collections.create(
            name="Article",
            properties=[
                # use DataType.TEXT (maps to "text") for unstructured text fields
                Property(name="title",    data_type=DataType.TEXT),
                # use DataType.DATE for date fields
                Property(name="batch_dt", data_type=DataType.DATE),
                # again TEXT for longer descriptions
                Property(name="desc",     data_type=DataType.TEXT),
            ],
            # omit vectorizer_config to use default Contextionary,
            # or swap in text2vec-openai/etc:
            # vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai()
        )

    # 3) Prepare rows for import
    data_rows = []
    for _, row in clean_news.iterrows():
        data_rows.append({
            "title":    row["title"],
            "batch_dt": row["batch_dt"],
            "desc":     row["desc"],
        })

    # 4) Batch-import into "Article"
    coll = client.collections.get("Article")
    with coll.batch.dynamic() as batch:
        for obj in data_rows:
            batch.add_object(properties=obj)
            # (batch will auto-flush when it reaches recommended size) :contentReference[oaicite:1]{index=1}

    print(f"✅ Indexed {len(data_rows)} articles into 'Article'") # :contentReference[oaicite:1]{index=1}

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 26673
ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 60', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: invalid date property \'batch_dt\' on class \'Article\': requires a string with a RFC3339 formatted date, but the given value is \'2025-04-18\'")'}
ERROR:weaviate-client:{'message': 'Failed to send 60 objects in a batch of 60. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}
ERROR:weaviate-client:{'message': 'Failed to send all objects in a batch of 60', 'error': 'WeaviateInsertManyAllFailedError("Every object failed during insertion. Here is the set of all errors: invalid date property \'batch_dt\' on class \'Article\': requires a string with a RFC3339 formatted date, but the given value is \'2025-04-18\'")'}
ERROR:weaviate-client:{'message': 'Failed to send 60 ob

✅ Indexed 1681 articles into 'Article'


In [47]:
import shutil
shutil.rmtree("/content/weaviate_data", ignore_errors=True)

In [52]:
import shutil
import weaviate
from weaviate.embedded import EmbeddedOptions
from weaviate.classes.config import Configure, Property, DataType

# 0) Wipe any old data so we start fresh
shutil.rmtree("/content/weaviate_data", ignore_errors=True)


emb = EmbeddedOptions(
    port                  = 8081,                        # HTTP port
    grpc_port             = 50051,                       # gRPC port
    version               = W_VERSION,                  # "1.26.5"
    persistence_data_path = "/content/weaviate_data",   # where to store on disk
    additional_env_vars   = {
        "ENABLE_MODULES": "text2vec-contextionary"
    }
)

# 2) Instantiate & start the embedded client
client = weaviate.WeaviateClient(embedded_options=emb)
client.connect()                            # launches the binary
assert client.is_ready(), "Weaviate failed to start"

# 3) Create the Article collection with contextionary vectorizer
if client.collections.exists("Article"):
    client.collections.delete("Article")

client.collections.create(
    name="Article",
    properties=[
        Property(name="title",    data_type=DataType.TEXT),
        Property(name="batch_dt", data_type=DataType.DATE),
        Property(name="desc",     data_type=DataType.TEXT),
    ],
    vectorizer_config=Configure.Vectorizer.text2vec_contextionary()
)

# 4) Batch-import your DataFrame rows
coll = client.collections.get("Article")
with coll.batch.dynamic() as batch:
    for _, row in clean_news.iterrows():
        batch.add_object({
            "title":    row["title"],
            "batch_dt": row["batch_dt"],   # ISO "YYYY-MM-DD"
            "desc":     row["desc"],
        })

# 5) Now hybrid / semantic queries work:
for hit in coll.query.hybrid("Pahalgam news", alpha=0.5, limit=5).objects:
    print(hit.properties["title"])

# 6) Clean up when done
client.close()


INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 39856


WeaviateStartUpError: Embedded DB did not start listening on port 8081 within 30 seconds

In [2]:
!pip install weaviate-client

  Using cached weaviate_client-4.14.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached validators-0.34.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached Authlib-1.3.1-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached grpcio_tools-1.71.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached grpcio_health_checking-1.71.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
Using cached weaviate_client-4.14.1-py3-none-any.whl (442 kB)
Using cached validators-0.34.0-py3-none-any.whl (43 kB)
Using cached Authlib-1.3.1-py2.py3-none-any.whl (223 kB)
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
Using cached grpcio_health_checking-1.71.0-py3-none-any.whl (18 kB)
Using cached grpcio_tools-1.71.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)


In [5]:
from weaviate.embedded import EmbeddedOptions

W_VERSION = "1.26.5"

client = weaviate.WeaviateClient(
    embedded_options = EmbeddedOptions(
        version=W_VERSION,                    # Weaviate core version
        persistence_data_path="/content/weaviate_data",  # where vectors live
    ),
)

client.connect()        # actually launches the embedded binary
print("Weaviate is up →", client.is_ready())

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.26.5/weaviate-v1.26.5-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 18634


Weaviate is up → True


In [78]:
client.close()

In [18]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import sqlite3
import weaviate
from google.colab import drive

# 1) load data
drive.mount('/content/drive')

DB_PATH = '/content/drive/Shareddrives/Welzin/Yana/YanaNewsProd.db'

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
conn    = sqlite3.connect(DB_PATH)
df      = pd.read_sql("SELECT title,batch_dt,desc FROM clean_news_new WHERE batch_dt='2025-04-23'", conn)

# 2) embed offline
embedder = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1")  # static 1024-dim, CPU-fast :contentReference[oaicite:2]{index=2}
vectors  = embedder.encode(df["desc"].tolist(), batch_size=32, show_progress_bar=True)

# 3) start Weaviate without any vectorizer module
client = weaviate.WeaviateClient(
  embedded_options=EmbeddedOptions(
    version=W_VERSION,
    persistence_data_path="/content/weaviate_data",
    env={"DEFAULT_VECTORIZER_MODULE": "none", "ENABLE_MODULES": ""}
  )
)
client.connect()

# 4) create a "dumb" class (we store our own vectors)
client.collections.delete_all()
schema = {
  "class": "CleanNews",
  "description": "Cleaned news articles (manual vectors)",
  "vectorizer": "none",
  "properties": [
    {"name":"title",   "dataType":["string"]},
    {"name":"batch_dt", "dataType":["date"]},
    {"name":"desc",    "dataType":["text"]},
  ]
}
client.collections.create_from_dict(schema)

# 5) ingest with explicit vector
with client.collections.get("CleanNews").batch as batcher:
  for obj, vec in zip(df.to_dict(orient="records"), vectors):
    batcher.add(
      data_object={
        "title":   obj["title"],
        "batch_dt": obj["batch_dt"],
        "desc":    obj["desc"]
      },
      class_name="CleanNews",
      vector=vec.tolist()
    )


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/670k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/125M [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

TypeError: EmbeddedOptions.__init__() got an unexpected keyword argument 'env'

In [79]:
import os
from weaviate.embedded import EmbeddedOptions
import weaviate

W_VERSION = "1.26.5"

# (Optional) turn off all built-in vectorizers
# os.environ["ENABLE_MODULES"] = ""
# os.environ["DEFAULT_VECTORIZER_MODULE"] = "none"

client = weaviate.WeaviateClient(
    embedded_options=EmbeddedOptions(
        version=W_VERSION,
        persistence_data_path="/content/weaviate_data",
    ),
)
client.connect()
print("Weaviate is up →", client.is_ready())

# now define & create your schema exactly as before:
client.collections.delete_all()

clean_news_schema = {
    "class":       "CleanNews",
    "description": "Cleaned news articles (manual vectors)",
    "vectorizer":  "none",      # since we’re pushing our own vectors
    "properties": [
      {"name": "title",   "dataType": ["string"]},
      {"name": "batch_dt","dataType": ["date"]},
      {"name": "desc",    "dataType": ["text"]},
    ],
}

client.collections.create_from_dict(clean_news_schema)

# then offline-embed & batch-push your vectors:
from sentence_transformers import SentenceTransformer
import pandas as pd, sqlite3

# 1) load
drive.mount('/content/drive')

DB_PATH = '/content/drive/Shareddrives/Welzin/Yana/YanaNewsProd.db'

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
conn    = sqlite3.connect(DB_PATH)
df      = pd.read_sql("SELECT title,batch_dt,desc FROM clean_news_new WHERE batch_dt = '2025-04-23'", conn)

# 2) embed
embedder = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1")
vectors  = embedder.encode(df["desc"].tolist(), batch_size=32, show_progress_bar=True)

# 3) push
with client.batch.dynamic() as batcher:
    for obj, vec in zip(df.to_dict(orient="records"), vectors):
        # original date string, e.g. "2025-04-18"
        date_only = obj["batch_dt"]
        # build a valid RFC3339 datetime
        iso_date  = f"{date_only}T00:00:00Z"

        batcher.add_object(
            collection="CleanNews",
            properties={
                "title":    obj["title"],
                "batch_dt": iso_date,         # <-- now valid
                "desc":     obj["desc"],
            },
            vector=vec.tolist(),
        )


INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 37780


Weaviate is up → True
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [80]:
from weaviate.classes.query import MetadataQuery

def hybrid_query_clean_news_with_score(query: str,
                                       alpha_val: float = 0.5,
                                       limit: int = 10):
    coll = client.collections.get("CleanNews")
    # embed your query as before
    query_vector = embedder.encode([query], show_progress_bar=False)[0].tolist()

    resp = coll.query.hybrid(
        query=query,
        vector=query_vector,
        alpha=alpha_val,
        limit=limit,
        # <-- ask for the normalized fused score in metadata
        return_metadata=MetadataQuery(score=True)
    )

    return resp.objects


In [83]:
results = hybrid_query_clean_news_with_score("Lionel messi", 0.5, 4)
if not results:
    print("No relevant docs found.")
else:
    for i, hit in enumerate(results, start=1):
        title = hit.properties["desc"]
        score = hit.metadata.score        # ← now this exists
        date  = hit.properties["batch_dt"]
        print(f"{i}. {title}  (score={score:.3f}, date={date})")


1. A Dhanbad-based woman found herself at the receiving end of the internet after she shared a video of her son appearing visibly distressed while doing his homework.

The video, shared by Saloni Agarwal on Instagram, shows the boy telling his mother that he is struggling to breathe. Instead of comforting him, Saloni is heard questioning him with a sarcastic remark: "You were breathing fine while playing earlier."

advertisement

The little boy responds, "Meri saans atak atak ke chal rahi hai (I'm unable to breathe properly)." Saloni, who seemed to assume he was faking it to avoid homework, went on to share the video online with a caption calling his behaviour "nautanki (drama)."

Take a look at Saloni's Instagram post here:

Watch the video here:

Saloni's attempt at humour didn't land well. The internet called her out for trivialising what looked like a genuine moment of distress. Social media users slammed her for documenting and posting the incident instead of offering support.

"T

In [63]:
# embed the same term you used above
qvec = embedder.encode(["Pahalgam"])[0].tolist()

results = client.collections.get("CleanNews") \
    .query.hybrid(
        query="",            # skip BM25
        vector=qvec,         # only vector
        alpha=1.0,
        limit=5
    )

for hit in results.objects:
    print(hit.properties["title"], f"(score={hit.score:.3f})")


AttributeError: 'Object' object has no attribute 'score'

In [64]:
coll = client.collections.get("CleanNews")

resp = coll.query.hybrid(
    query="pahalgam",
    vector=embedder.encode(["pahalgam"])[0].tolist(),
    alpha=0.5,
    limit=5
)

# dump out the full GraphQL-style dict
import pprint
pprint.pprint(resp.__dict__)   # or print(resp) if __repr__ is JSON-like


{'objects': [Object(uuid=_WeaviateUUIDInt('06365fb5-41e2-496e-8538-575d909a089b'),
                    metadata=MetadataReturn(creation_time=None,
                                            last_update_time=None,
                                            distance=None,
                                            certainty=None,
                                            score=None,
                                            explain_score=None,
                                            is_consistent=None,
                                            rerank_score=None),
                    properties={'batch_dt': datetime.datetime(2025, 4, 23, 0, 0, tzinfo=datetime.timezone.utc),
                                'desc': 'Vicky Kaushal took to Instagram and '
                                        'wrote, “Can’t imagine the pain of the '
                                        'family members who lost their loved '
                                        'ones in the absolutely inhu